In [28]:
from fred import Fred
import numpy as np
import pandas as pd
from datetime import datetime

# FRED API key
key = '1c08c3ed2d66cacc1dfa6574865b2bdd'

## Getting Data from FRED ##
fr = Fred(api_key = key, response_type = 'df')

# Setting data parameters
start = datetime(1995, 1, 1)
end = datetime(2010, 1, 1)

params = {'observation_start': start.strftime('%Y-%m-%d'),
          'observation_end': end.strftime('%Y-%m-%d')}

# Load list of data series
fseries = pd.read_csv("FRED_data_series.csv")

In [30]:
def growthRate(series):
    return np.exp(np.diff(np.log(series))) - 1

def levelShrink(dates, series):
    ret = []
    for start, end in zip(dates[:-1], dates[1:]):
        batch = series.loc[(series.date >= start) & (series.date < end), 'value']
        shrink = np.prod(1 + batch) - 1
        ret.append(shrink)
    return ret

def percentShrink(dates, series):
    ret = []
    for start, end in zip(dates[:-1], dates[1:]):
        batch = series.loc[(series.date >= start) & (series.date < end), 'value']
        shrink = np.sum(np.diff(batch))
        ret.append(shrink)
    return ret

def dataGetter(code, dates, method):
    series = fr.series.observations(code, params = params)
    if(method):
        series = percentShrink(dates, series)
    else:
        series = pd.DataFrame({'date' : series.date[1:], 'value': growthRate(series.value)})
        series = levelShrink(dates, series)
    return series

In [31]:
# Loading GDP
gdp = fr.series.observations("GDP", params = params)
gdp.drop(labels = ['realtime_end', 'realtime_start'], axis = 1, inplace = True)
gdp.rename(columns = {'value' : 'gdp'}, inplace = True)

# Retrieving data from Fred
n = fseries.size
dates = gdp.date

data = pd.DataFrame({'date' : dates[1:]})

for i in range(n):
    code = fseries.loc[i, 'file']
    method = fseries.loc[i, 'method']
    data[code] = dataGetter(code, dates, method)
data.head()

C:\Users\Davis Berlind\Miniconda3\lib\site-packages\fred\helpers\__init__.py:36: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  frame = frame.convert_objects(convert_numeric=True)


HTTPError: HTTP Error 429: Too Many Requests

In [32]:
data

,date,CAPUTLN2121S,CAPUTLG3342S,CAPUTLG3341S,CAPUTLHITEK2S,CAPUTLB5610CS,CAPUTLGMFDS,CAPUTLG3364T9S,CAPUTLG33611S,CAPUTLG334S,...,IPG32411S,IPG3254S,IPN325211S,IPG3261S,IPN311611T3PS,IPN311615S,IPN32211S,IPG3221S,IPN32411RS,IPG32521S
1,1995-04-01,-2.4331,-1.9631,-1.2589,-0.0935,-0.5988,-0.7462,-0.2588,0.3631,0.0082,...,0.000540,0.006624,-0.065233,-0.017369,0.034972,0.026526,0.007642,0.007035,-0.084953,-0.069742
2,1995-07-01,0.4888,-0.2481,3.8056,0.5344,0.1027,-0.0593,1.3697,-1.8214,0.8486,...,-0.013667,-0.006229,-0.038181,-0.018734,0.002790,0.030211,0.016200,-0.006689,-0.017896,-0.020650
3,1995-10-01,1.7286,0.2761,2.9338,2.1872,-1.2971,1.6680,-1.9346,6.9107,1.7711,...,0.018865,0.038669,-0.023681,0.014410,-0.056670,-0.030038,-0.038066,-0.033817,0.069591,-0.029586
4,1996-01-01,-2.6435,-3.1117,3.2540,-0.5490,-0.8910,-0.1918,-0.9580,0.3289,-0.9137,...,-0.014172,0.025013,-0.029427,0.002936,0.005826,0.006122,-0.046972,-0.032201,-0.027830,-0.038751
5,1996-04-01,9.5144,1.6675,-1.1478,-0.4538,1.2536,0.0209,4.4797,-7.1704,0.0148,...,0.009725,-0.004880,0.016705,0.009800,-0.056673,-0.016713,-0.028916,-0.013523,-0.119782,0.020098
6,1996-07-01,2.1718,-0.0313,1.9611,-1.0254,0.8677,0.6262,1.6908,1.7531,-0.7526,...,0.003924,0.023503,0.013119,0.025573,-0.051989,-0.022137,-0.000031,0.007414,0.067216,0.015244
7,1996-10-01,-2.3184,-0.1727,-0.2036,-0.3350,-0.1120,0.1265,0.6278,-4.1149,-0.3089,...,0.013997,0.007079,0.009151,0.020780,0.075726,0.015264,-0.005486,-0.010709,-0.034430,0.013653
8,1997-01-01,-0.1278,0.8460,-3.7948,-1.4573,0.5798,0.5412,0.8591,6.9320,-0.9501,...,0.006055,0.020503,0.006540,0.011017,-0.045997,0.021119,-0.037732,0.023429,0.001147,0.023980
9,1997-04-01,2.4156,0.7043,0.7794,2.1963,1.2610,1.1258,0.9317,2.5304,1.9617,...,0.005100,0.002151,0.043888,0.012732,0.072167,0.065773,0.046338,-0.012796,-0.101975,0.036218
10,1997-07-01,0.8561,1.3881,1.0119,1.3291,-0.3583,0.4553,-0.4057,5.2226,0.9935,...,0.021555,0.013481,0.020366,0.003273,-0.000309,0.046071,-0.030887,-0.016892,0.148782,0.014371
